In [6]:
import pandas as pd
import geopandas as gpd
import os

# === BASE DIRECTORY ===
base_dir = r"C:\Users\natda\OneDrive - Northeastern University\Desktop\NatDave\Academics\PhD_NU\RESEARCH\Traffic_Stress\Boston"

# === FILE PATHS ===
roads_path = os.path.join(base_dir, "street_network.shp")

# === LOAD SHAPEFILE ===
roads = gpd.read_file(roads_path)

# === CONVERT RELEVANT COLUMNS TO NUMERIC ===
numeric_cols = ['SPEED', 'qDirLanes2', 'qNoAccess', 'qExclude', 'BL_WIDTH', 'BL_REACH', 'PARKALONG', 'ILLPARKING', 'StOperNEU', 'ADT_2025', 'FEDERALFUN', 'BL_BLOCK']
roads[numeric_cols] = roads[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [7]:
# === FUNCTION TO CALCULATE LTS_2025b ===
def calculate_LTS_2025b(row):
    """Calculate LTS_2025b based on road characteristics."""
    
    protected = row['qProtected']
    no_access = row['qNoAccess']
    exclude = row['qExclude']
    speed = row['SPEED']
    dir_lanes = row['qDirLanes2']
    bl_width = row['BL_WIDTH']
    bl_reach = row['BL_REACH']
    parkalong = row['PARKALONG']
    # illparking = row['ILLPARKING']
    bike_type = row['bike_type2']
    st_oper_neu = row['StOperNEU']
    adt = row['ADT_2025']
    fed_fun = row['FEDERALFUN']
    wide_oneway = row['WIDE_1WAY']
    bl_block = row['BL_BLOCK']  

    lane_infra = ["BL", "BL_LEFT",  "BL_BUF", "BL_BUF_LEFT", "SBL_BL", "MIX_CONTRA", "BL_PK_BUS_BL"]
    sep_infra = ["SUP", "SUP_NAT", "SUP_MINOR", "SBL", "SBL_LEFT", "MIX_SCONTRA", "CARFREE"]
    mix_infra = ["SBL_MIX", "BL_MIX", "SLM", "BUS_BL", "BUS_BL_LEFT", "SLMTC"]


    if no_access in (1, 98, 99):
        return 6  # freeway (ramps)
    elif exclude in (1, 5) or (bike_type and bike_type in ("WALK_YR_BIKE")):
        return 5  # cemetary, private property, peds only, etc.
    elif fed_fun == 0:
        return 1  # Parks, alleys, bike paths, streets with almost no traffic
    elif bike_type in sep_infra:
        return 1  # Separated from traffic
    elif protected == 1:
        return 1 # Separated from traffic
    

    elif ((bl_width >= 4) or bike_type in lane_infra) and (parkalong == 0):
        # Conventional bike lanes not adjacent to parking
        if (
            (dir_lanes >= 3 and speed > 38.5) or 
            (dir_lanes == 2 and speed > 43.5 and bl_width < 6) or
            (dir_lanes <= 1 and speed > 48.5 and bl_width < 6)
        ):
            return 4
        elif (dir_lanes >= 3) or (speed > 38.5):
            return 3
        elif (
            (dir_lanes == 2) or
            (speed > 33.5) or
            (dir_lanes <= 1 and bl_width < 6) or
            (dir_lanes <= 1 and speed > 28.5)
        ):
            return 2
        elif (dir_lanes <= 1 and bl_width >= 6):
            return 1
        else:
            return 98 # For other cases that do not meet conditions


    elif (bl_width >= 4 or bike_type in lane_infra) and (parkalong == 1): # and (bl_reach >= 12):
        # Conventional bike lanes adjacent to parking
        if (dir_lanes <= 1 and speed <= 28.5 and bl_reach >= 15):
            return 1
        elif (
            (dir_lanes <= 1 and speed <= 38.5 and bl_reach >= 15) or
            (dir_lanes <= 1 and speed <= 33.5 and bl_reach < 15) or
            (dir_lanes >= 2 and st_oper_neu == 1 and speed <= 28.5 and bl_reach >= 15) or
            (dir_lanes == 2 and speed <= 28.5 and bl_reach >= 15)
        ):
            return 2
        else:
            return 3


    elif (
        (bl_width < 4) or 
        ((bl_width >= 4) and (bl_block == 1)) or 
        ((bl_width >= 4) and (parkalong > 0) and (bl_reach < 12)) or
        bike_type in mix_infra
    ):
        # Mixed traffic conditions
        if (
            # 3+ thru lanes per direction
            (dir_lanes >= 3 and speed > 28.5) or

            # 2 thru lanes per direction
            (dir_lanes == 2 and adt > 8_000 and speed > 28.5) or 
            (dir_lanes == 2 and adt <= 8_000 and speed > 38.5) or

            # Narrow one-way single-lane street
            (dir_lanes <= 1 and st_oper_neu == 1 and adt > 1_000 and speed > 38.5) or
            (dir_lanes <= 1 and st_oper_neu == 1 and adt > 600 and speed > 43.5) or

            # Two-way 1+1 per direction with CL or wide one-way street
            ((dir_lanes == 1 or wide_oneway == 1) and adt > 1_500 and speed > 38.5) or
            ((dir_lanes == 1 or wide_oneway == 1) and adt > 1_000 and speed > 43.5) or

            # Unlaned two-way streets
            (dir_lanes == 0 and adt > 3_000 and speed > 38.5) or
            (dir_lanes == 0 and adt > 1_500 and speed > 43.5)
        ):
            return 4

        elif (
            # 2+ thru lanes per direction
            dir_lanes >= 2 or

            # Narrow one-way single-lane street
            (dir_lanes <= 1 and st_oper_neu == 1 and adt > 1_000 and speed > 23.5) or
            (dir_lanes <= 1 and st_oper_neu == 1 and adt > 600 and speed > 33.5) or
            (dir_lanes <= 1 and st_oper_neu == 1 and speed > 38.5) or

            # Two-way 1+1 per direction with CL or wide one-way street
            ((dir_lanes == 1 or wide_oneway == 1) and adt > 1_500 and speed > 23.5) or
            ((dir_lanes == 1 or wide_oneway == 1) and adt > 1_000 and speed > 33.5) or
            ((dir_lanes == 1 or wide_oneway == 1) and speed > 38.5) or

            # Unlaned two-way streets
            (dir_lanes == 0 and adt > 3_000 and speed > 28.5) or
            (dir_lanes == 0 and adt > 750 and speed > 33.5) or
            (dir_lanes == 0 and speed > 38.5)
        ):
            return 3

        elif (
            # Narrow one-way single-lane street
            (dir_lanes <= 1 and st_oper_neu == 1 and adt > 600) or
            (dir_lanes <= 1 and st_oper_neu == 1 and speed > 28.5) or

            # Two-way 1+1 per direction with CL or wide one-way street
            ((dir_lanes == 1 or wide_oneway == 1) and adt > 1_000) or
            ((dir_lanes == 1 or wide_oneway == 1) and speed > 28.5) or

            # Unlaned two-way streets
            (dir_lanes == 0 and adt > 1_500) or
            (dir_lanes == 0 and speed > 28.5)
        ):
            return 2

        elif (
            # Narrow one-way single-lane street
            (dir_lanes <= 1 and st_oper_neu == 1) or

            # Two-way 1+1 per direction with CL or wide one-way street
            (dir_lanes == 1 or wide_oneway == 1) or

            # Unlaned two-way streets
            dir_lanes == 0
        ):
            return 1
        else:
            return 99  # For other cases that do not meet conditions

    else:
        return 96  # Default return for other cases

# === CALCULATE LTS_2025b FOR ROADS ===
roads['LTS_2025b'] = None  # Initialize the column
roads['LTS_2025b'] = roads.apply(calculate_LTS_2025b, axis=1)

In [8]:
# Set LTS_2025b to -1 where StOperNEU is 1 or 11
roads.loc[roads['StOperNEU'].isin([1, 11]), 'LTS_2025b'] = -1

In [9]:
# === SAVE THE UPDATED ROADS SHAPEFILE ===
roads.to_file(roads_path, driver="ESRI Shapefile")
 
# Output summary
print("LTS_2025b calculation complete.")
print(roads['LTS_2025b'].value_counts(dropna=False))

LTS_2025b calculation complete.
LTS_2025b
 1    14351
-1     9076
 5     6093
 3     3713
 6     1729
 2     1423
 4      411
Name: count, dtype: int64


In [10]:
roads['bike_type2'].value_counts(dropna=False)

bike_type2
None            30631
BL               1421
SUP              1276
SLM              1126
SBL               833
SUP_MINOR         530
BL_MIX            216
BL_BUF            211
BUS_BL            103
WALK_YR_BIKE       79
SUP_NAT            75
SLMTC              54
SBL_BL             52
MIX_CONTRA         38
SBL_MIX            33
BL_LEFT            30
BL_PK_BUS_BL       29
CARFREE            20
MIX_SCONTRA        11
SBL_LEFT           11
BUS_BL_LEFT         8
BL_BUF_LEFT         4
                   3
CFBS                2
Name: count, dtype: int64